In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import random
import pickle
from sklearn.metrics import classification_report
# функции из skimage для редактирования картинок
from skimage import transform
from skimage import exposure # повышение контрастности
from skimage import io
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow

In [ ]:
input_shape = (32, 32, 3)
chanDim = -1

In [ ]:
inputs = tf.keras.layers.Input(shape=input_shape)

# first set of CONV -> BATCH_NORM -> POOL
# Первым аргументом на Conv2D() стоит 8, это значит, что получится 8 карт признаков https://colab.research.google.com/drive/1IBZGOZa4yM1-Q4Mye1a1XTUG78doRr9o#scrollTo=BG4hdaOYLDaI&line=8&uniqifier=1
# relu - ускоряет обучение сети
conv1 = tf.keras.layers.Conv2D(8, (5, 5), padding='same', activation='relu')(inputs)
batch_normalization1 = tf.keras.layers.BatchNormalization(axis=chanDim)(conv1)
pooling1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(batch_normalization1)
conv2 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(pooling1)
batch_normalization2 = tf.keras.layers.BatchNormalization(axis=chanDim)(conv2)
conv3 = tf.keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(batch_normalization2)
batch_normalization3 = tf.keras.layers.BatchNormalization(axis=chanDim)(conv3)
pooling2 = tf.keras.layers.MaxPooling2D((2, 2))(batch_normalization3)

# second set of CONV -> BATCH_NORM -> POOL
conv4 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(pooling2)
batch_normalization4 = tf.keras.layers.BatchNormalization(axis=chanDim)(conv4)
conv5 = tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu')(batch_normalization4)
batch_normalization5 = tf.keras.layers.BatchNormalization(axis=chanDim)(conv5)
pooling3 = tf.keras.layers.MaxPooling2D((2, 2))(batch_normalization5)

# 
flatten1 = tf.keras.layers.Flatten()(pooling3)
dense1 = tf.keras.layers.Dense(128, activation='relu')(flatten1)
batch_normalization6 = tf.keras.layers.BatchNormalization(axis=chanDim)(dense1)
dropout1 = tf.keras.layers.Dropout(0.5)(batch_normalization6)
flatten2 = tf.keras.layers.Flatten()(dropout1)
dense2 = tf.keras.layers.Dense(128, activation='relu')(flatten2)
batch_normalization7 = tf.keras.layers.BatchNormalization(axis=chanDim)(dense2)
dropout2 = tf.keras.layers.Dropout(0.5)(batch_normalization7)

out = tf.keras.layers.Dense(7, activation='softmax')(dropout2)

In [ ]:
# Подготовка датасета
path = 'drive/MyDrive/Colab Notebooks/Course/multiple_classification'

with open(f'{path}/dataNew.pickle', 'rb') as f:
    data = pickle.load(f)
with open(f'{path}/labels.pickle', 'rb') as f:
    labels = pickle.load(f)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.15,
                                                  random_state=42)
# Узнаём сколько изображений каждого класса
class_totals = trainY.sum(axis=0)
print(f'Amount of images for each class: {class_totals}')

# значения весов
class_weight = class_totals.max() / class_totals
print(f'Class weights: {class_weight // 1}')

# То же самое проделываем и для тестового датасета
test_class_totals = testY.sum(axis=0)
test_class_weight = test_class_totals.max() / test_class_totals
f = test_class_totals.argmin()

Amount of images for each class: [1759 1851  658  548  938  333 1282]
Class weights: [1. 1. 2. 3. 1. 5. 1.]


In [ ]:
model = tf.keras.Model(inputs=inputs, outputs=out)

In [ ]:
EPOCHS = 9
LEARNING_RATE = 1e-3
BATCH_SIZE = 64

target_names= ["Main road", "Give way", "Stop", "Traffic is prohibited", "entry is forbidden", "Rough road", "RoadWork"]
num_labels = len(target_names)

opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, decay=LEARNING_RATE / (EPOCHS * 0.5))

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
# model.summary() показывает какие слои есть у модели, формы тензоров модели, количество весовых коэффициентов
# model.summary()

In [ ]:
# ImageDataGenerator - изменяет исходное изображение. Произвольно вращает картинку на 5 градусов
# zoom_range - увеличение изображения
# width_shift_range - смещение по ширине. Изображение просто будет сдвигаться (влево, вправо). С height_shift_range то же самое только, смещение происходит вверх и вниз
# shear_range - изменение цвета в каждом канале изображения (изображение 3-х цветное). Положительно повлияет на распознавание в разных условиях освещения
aug = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=5,
    zoom_range=0.05,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest'
)

class_Weight = {0: 1.06027728,
                1: 1.,
                2: 2.84627832,
                3: 3.46259843,
                4: 2.01028571,
                5: 5.5664557,
                6: 1.46461282}

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
]

In [ ]:
# aug.flow() начинает генрироваться картинки
# batch_size - размер картинок, который за раз даётся на вход сети
# steps per epoch - сколько шагов в каждой эпохе
# эпоха - отрезок времени в течение которого все данные прогоняются через сеть
# shuffle=True нужен для того, чтобы данные перемешивались после каждой эпохи
# class_weight нужен для того, чтобы указать, какой весовой коэффициент надо учитывать больше при обучении

# H = model.fit(aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
#                         validation_data=(testX, testY),
#                         steps_per_epoch=trainX.shape[0] // BATCH_SIZE,
#                         shuffle=True,
#                         class_weight=class_Weight,
#                         callbacks=callbacks)


model.fit(aug.flow(trainX, trainY, batch_size=BATCH_SIZE),
            validation_data=(testX, testY),
            steps_per_epoch=trainX.shape[0] // BATCH_SIZE,
            epochs=EPOCHS,
            shuffle=True,
            class_weight=class_Weight,
            callbacks=callbacks)

Epoch 1/9
115/115 [==============================] - 12s 38ms/step - loss: 3.7947 - accuracy: 0.3285 - val_loss: 2.1033 - val_accuracy: 0.1322
Epoch 2/9
115/115 [==============================] - 4s 33ms/step - loss: 0.8561 - accuracy: 0.8269 - val_loss: 2.4996 - val_accuracy: 0.2621
Epoch 3/9
115/115 [==============================] - 4s 32ms/step - loss: 0.3541 - accuracy: 0.9328 - val_loss: 2.0180 - val_accuracy: 0.2621
Epoch 4/9
115/115 [==============================] - 4s 32ms/step - loss: 0.2115 - accuracy: 0.9637 - val_loss: 1.5044 - val_accuracy: 0.5503
Epoch 5/9
115/115 [==============================] - 4s 32ms/step - loss: 0.1300 - accuracy: 0.9780 - val_loss: 0.6194 - val_accuracy: 0.7909
Epoch 6/9
115/115 [==============================] - 4s 32ms/step - loss: 0.0983 - accuracy: 0.9852 - val_loss: 0.0508 - val_accuracy: 0.9892
Epoch 7/9
115/115 [==============================] - 4s 33ms/step - loss: 0.0728 - accuracy: 0.9875 - val_loss: 0.0121 - val_accuracy: 0.9969
Epoch

In [ ]:
model.save('drive/MyDrive/Colab Notebooks/Course/multiple_classification/AdrNet.h5')

In [ ]:
model.evaluate(trainX, trainY, batch_size=BATCH_SIZE)

116/116 [==============================] - 0s 3ms/step - loss: 0.0108 - accuracy: 0.9974


[0.0108237499371171, 0.9974216222763062]

In [ ]:
model.predict(testX[:1])

array([[2.0897419e-06, 9.9984205e-01, 6.7901739e-05, 1.3310544e-05,
        2.7464703e-05, 3.8406761e-05, 8.7644439e-06]], dtype=float32)